In [1]:
import pandas as pd
# import nltk.data
import nltk
from collections import OrderedDict, Counter
from tqdm import tqdm
import regex as re
# import re, string
import itertools
import pickle

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1000)


# text-to-sentence

In [210]:
def clean_text(input_text):
    # preprocess text where sentences end with no space after the full stop
    # e.g., "This a sentence.This another one.""
    text = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', input_text)
    # Remove new lines
    text = text.replace('\n', ' ').strip()
    # Remove URL links
    text = re.sub(r'http\S+', '', text)
    # Remove patterns like "Validity of ASD subtypes: \n\n</ref>""
    text = re.sub(r'\.[\s\S\][^.]*?<\/ref>', '. ', text)
    # Remove e.g., '| |- |colspan=2|', '|- |rowspan=12|', '| - |- |rowspan=4|'
    text = re.sub(r'(\|[\-\s]*)|(rowspan=\d+)|(colspan=\d+)', ' ', text)
    # Remove images in <gallery ...> ... </gallery>, <imagemap> ... </imagemap>
    text = re.sub(r'<([\w\s\W]*?)>(.*?)<(\/?)([\w\s\W]*?)>', '', text)
    # Remove html tags
    text = re.sub(r'<(\/?)([\w\s\W]*?)(\/?)>', '', text)
    
    return text.strip()

def articles_to_sentences(articles_in_json):
    df_med_conditions = pd.read_json(articles_in_json, lines=True)
    
    # Don't extract sentences from the following articles
    excl_articles = ["Wikipedia:Administrators' noticeboard/IncidentArchive1003",
                     "Wikipedia:Article",
                     "Wikipedia:Bot",
                     "Wikipedia:Deletion review/Log/2015 February 21",
                     "Wikipedia:Featured article candidates/Acne vulgaris/archive2",
                     "Wikipedia:Fringe theories/Noticeboard/Archive 57",
                     "Wikipedia:Help desk/Archives/2016 August 20",
                     "Wikipedia:Manual",
                     "Wikipedia:Teahouse/Questions/Archive 671",
                     "Wikipedia:Templates",
                     "Wikipedia:Village"
                    ]

    data = []
    for index, row in tqdm(df_med_conditions.iterrows()):                
        if any(x in row['title'] for x in excl_articles):
            continue
        
        title = row['title']
        text = clean_text(row['parsed_text'])

        for i, sentence in enumerate(nltk.tokenize.sent_tokenize(text)):
            # Remove sentences which only contain one word
            if len(sentence.split()) == 1:
                continue
            # Remove sentences which only contain digits and punctuations
            if not any(c.isalpha() for c in sentence):
                continue
                
            data.append(OrderedDict({"source_page": title, "sent_id":i+1 ,"sentence": sentence}))
    
    # Create dataframe objects
    data = pd.DataFrame(data)

    return data

In [211]:
articles_in_json = './extracted/articles/medical_conditions_V4.json'

In [212]:
articles = articles_to_sentences(articles_in_json)

8677it [01:27, 99.40it/s] 


In [217]:
articles

,source_page,sent_id,sentence
0,Autism,1,"Autism is a developmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior."
1,Autism,2,Autism's symptoms result from maturation-related changes in various systems of the brain.
2,Autism,3,"Some studies have reported diagnoses of autism in children due to a loss of language or social skills, as opposed to a failure to make progress, typically from 15 to 30 months of age."
3,Autism,4,No known medication relieves autism's core symptoms of social and communication impairments.
4,Autism,5,"Treatment approaches have little empirical support in quality-of-life contexts, and many programs focus on success measures that lack predictive validity and real-world relevance."
...,...,...,...
350757,Hepatosplenic T-cell lymphoma,25,Autologous bone marrow transplantation is currently being investigated in the treatment of hepatosplenic lymphoma.
350758,Hepatosplenic T-cell lymphoma,26,Allogeneic bone marrow transplant has been proven to attain remission for over five years and possibly cure hepatosplenic lymphoma with a number of patients surviving if they can make it to Allogenic transplant.
350759,Hepatosplenic T-cell lymphoma,27,"This lymphoma is rare, comprising less than 5% of all cases, and is most common in young adults and adolescents."
350760,Hepatosplenic T-cell lymphoma,28,A distinct male gender preference has been described.


In [218]:
articles.to_csv('./sentences_and_ngrams/medcon_sentences.csv', sep=';', index=False)

# Extract n-grams

## pipeline n-gram extraction

In [17]:
def remove_punctuation(sentence):
    sent = re.sub(r"[^\s\p{L}\p{N}'-]|(?<=(^|\s))'|'(?=($|\s))|(?<=(^|\s))-|-(?=($|\s))", '', sentence)
    return sent

def extract_ngrams(sentence, num):
    new_sentence = remove_punctuation(sentence.lower())
    tokens = nltk.word_tokenize(new_sentence)
    tokens_POS = nltk.pos_tag(tokens)
    n_grams = nltk.ngrams(tokens_POS, num)

    return list(n_grams)

def group_ngrams(filtered_ngram_dict):
    ngrams_grouped = OrderedDict()
    for item in filtered_ngram_dict.items():
        #print(item[0]) #keys, e.g.: (('disorder', 'NN'),)
        #print(item[1]) #values (freq), e.g.: 11386
        ngrams = []
        for gram in item[0]:
            #print(gram[0]) #token
            #print(gram[1]) #POS
            ngrams.append(gram[0])
        ngrams_grouped[" ".join(ngrams)] = item[1]
    return ngrams_grouped

In [223]:
medcon_sentences = pd.read_csv('./sentences_and_ngrams/medcon_sentences.csv', sep=';')
medcon_sentences

medcon_ngrams = []
for index, row in tqdm(medcon_sentences.iterrows()):
    one_grams = extract_ngrams(row['sentence'], 1)
    bi_grams = extract_ngrams(row['sentence'], 2)
    tri_grams = extract_ngrams(row['sentence'], 3)
    four_grams = extract_ngrams(row['sentence'], 4)
    if index <10:
        print(row['sentence'])
        print("uni_grams:\n", one_grams)
        print("bi_grams:\n", bi_grams)
        print("tri_grams:\n", tri_grams)
        print("quandri_grams:\n", four_grams)
        print(" ")
    
    medcon_ngrams.append(OrderedDict({"unigrams": one_grams,
                                      "bigrams": bi_grams,
                                      "trigrams": tri_grams,
                                      "quandrigrams": four_grams
                                     }))

medcon_ngrams = pd.DataFrame(medcon_ngrams)
medcon_df = pd.concat([medcon_sentences, medcon_ngrams], axis=1)

46it [00:00, 175.17it/s]

Autism is a developmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.
uni_grams:
 [(('autism', 'NN'),), (('is', 'VBZ'),), (('a', 'DT'),), (('developmental', 'JJ'),), (('disorder', 'NN'),), (('characterized', 'VBN'),), (('by', 'IN'),), (('difficulties', 'NNS'),), (('with', 'IN'),), (('social', 'JJ'),), (('interaction', 'NN'),), (('and', 'CC'),), (('communication', 'NN'),), (('and', 'CC'),), (('by', 'IN'),), (('restricted', 'VBN'),), (('and', 'CC'),), (('repetitive', 'JJ'),), (('behavior', 'NN'),)]
bi_grams:
 [(('autism', 'NN'), ('is', 'VBZ')), (('is', 'VBZ'), ('a', 'DT')), (('a', 'DT'), ('developmental', 'JJ')), (('developmental', 'JJ'), ('disorder', 'NN')), (('disorder', 'NN'), ('characterized', 'VBN')), (('characterized', 'VBN'), ('by', 'IN')), (('by', 'IN'), ('difficulties', 'NNS')), (('difficulties', 'NNS'), ('with', 'IN')), (('with', 'IN'), ('social', 'JJ')), (('social', 'JJ'), ('interaction', 'NN')), 

350762it [22:00, 265.59it/s]


In [224]:
medcon_df.to_csv('./sentences_and_ngrams/medcon_sentences_ngrams.csv', sep=';', index=False)

In [228]:
medcon_df

,source_page,sent_id,sentence,unigrams,bigrams,trigrams,quandrigrams
0,Autism,1,"Autism is a developmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.","[((autism, NN),), ((is, VBZ),), ((a, DT),), ((developmental, JJ),), ((disorder, NN),), ((characterized, VBN),), ((by, IN),), ((difficulties, NNS),), ((with, IN),), ((social, JJ),), ((interaction, NN),), ((and, CC),), ((communication, NN),), ((and, CC),), ((by, IN),), ((restricted, VBN),), ((and, CC),), ((repetitive, JJ),), ((behavior, NN),)]","[((autism, NN), (is, VBZ)), ((is, VBZ), (a, DT)), ((a, DT), (developmental, JJ)), ((developmental, JJ), (disorder, NN)), ((disorder, NN), (characterized, VBN)), ((characterized, VBN), (by, IN)), ((by, IN), (difficulties, NNS)), ((difficulties, NNS), (with, IN)), ((with, IN), (social, JJ)), ((social, JJ), (interaction, NN)), ((interaction, NN), (and, CC)), ((and, CC), (communication, NN)), ((communication, NN), (and, CC)), ((and, CC), (by, IN)), ((by, IN), (restricted, VBN)), ((restricted, VBN), (and, CC)), ((and, CC), (repetitive, JJ)), ((repetitive, JJ), (behavior, NN))]","[((autism, NN), (is, VBZ), (a, DT)), ((is, VBZ), (a, DT), (developmental, JJ)), ((a, DT), (developmental, JJ), (disorder, NN)), ((developmental, JJ), (disorder, NN), (characterized, VBN)), ((disorder, NN), (characterized, VBN), (by, IN)), ((characterized, VBN), (by, IN), (difficulties, NNS)), ((by, IN), (difficulties, NNS), (with, IN)), ((difficulties, NNS), (with, IN), (social, JJ)), ((with, IN), (social, JJ), (interaction, NN)), ((social, JJ), (interaction, NN), (and, CC)), ((interaction, NN), (and, CC), (communication, NN)), ((and, CC), (communication, NN), (and, CC)), ((communication, NN), (and, CC), (by, IN)), ((and, CC), (by, IN), (restricted, VBN)), ((by, IN), (restricted, VBN), (and, CC)), ((restricted, VBN), (and, CC), (repetitive, JJ)), ((and, CC), (repetitive, JJ), (behavior, NN))]","[((autism, NN), (is, VBZ), (a, DT), (developmental, JJ)), ((is, VBZ), (a, DT), (developmental, JJ), (disorder, NN)), ((a, DT), (developmental, JJ), (disorder, NN), (characterized, VBN)), ((developmental, JJ), (disorder, NN), (characterized, VBN), (by, IN)), ((disorder, NN), (characterized, VBN), (by, IN), (difficulties, NNS)), ((characterized, VBN), (by, IN), (difficulties, NNS), (with, IN)), ((by, IN), (difficulties, NNS), (with, IN), (social, JJ)), ((difficulties, NNS), (with, IN), (social, JJ), (interaction, NN)), ((with, IN), (social, JJ), (interaction, NN), (and, CC)), ((social, JJ), (interaction, NN), (and, CC), (communication, NN)), ((interaction, NN), (and, CC), (communication, NN), (and, CC)), ((and, CC), (communication, NN), (and, CC), (by, IN)), ((communication, NN), (and, CC), (by, IN), (restricted, VBN)), ((and, CC), (by, IN), (restricted, VBN), (and, CC)), ((by, IN), (restricted, VBN), (and, CC), (repetitive, JJ)), ((restricted, VBN), (and, CC), (repetitive, JJ), (behavior, NN))]"
1,Autism,2,Autism's symptoms result from maturation-related changes in various systems of the brain.,"[((autism, NN),), (('s, POS),), ((symptoms, NNS),), ((result, VBP),), ((from, IN),), ((maturation-related, JJ),), ((changes, NNS),), ((in, IN),), ((various, JJ),), ((systems, NNS),), ((of, IN),), ((the, DT),), ((brain, NN),)]","[((autism, NN), ('s, POS)), (('s, POS), (symptoms, NNS)), ((symptoms, NNS), (result, VBP)), ((result, VBP), (from, IN)), ((from, IN), (maturation-related, JJ)), ((maturation-related, JJ), (changes, NNS)), ((changes, NNS), (in, IN)), ((in, IN), (various, JJ)), ((various, JJ), (systems, NNS)), ((systems, NNS), (of, IN)), ((of, IN), (the, DT)), ((the, DT), (brain, NN))]","[((autism, NN), ('s, POS), (symptoms, NNS)), (('s, POS), (symptoms, NNS), (result, VBP)), ((symptoms, NNS), (result, VBP), (from, IN)), ((result, VBP), (from, IN), (maturation-related, JJ)), ((from, IN), (maturation-related, JJ), (changes, NNS)), ((maturation-related, JJ), (changes, NNS), (in, IN)), ((changes, NNS), (in

In [18]:
medcon_sentences = pd.read_csv('./sentences_and_ngrams/medcon_sentences.csv', sep=';')

medcon_df = pd.read_csv('./sentences_and_ngrams/medcon_sentences_ngrams.csv', sep=';')
columns_to_convert = ['unigrams',
                      'bigrams',
                      'trigrams',
                      'quandrigrams'
                     ]
# Convert selected columns into lists
for i,column in enumerate(columns_to_convert):
    medcon_df[column] = medcon_df[column].apply(lambda x: eval(x))

medcon_ngrams_filtered = []

# Frequencies of all the n-grams
unigrams_freq = nltk.FreqDist(list(itertools.chain(*medcon_df['unigrams'].tolist())))
bigrams_freq = nltk.FreqDist(list(itertools.chain(*medcon_df['bigrams'].tolist())))
trigrams_freq = nltk.FreqDist(list(itertools.chain(*medcon_df['trigrams'].tolist())))
quandrigrams_freq = nltk.FreqDist(list(itertools.chain(*medcon_df['quandrigrams'].tolist())))

# Filter n-grams per sentence based on POS patterns
for index, row in tqdm(medcon_df.iterrows()):
    filtered_unigrams = OrderedDict()
    filtered_bigrams = OrderedDict()
    filtered_trigrams = OrderedDict()
    filtered_quandrigrams = OrderedDict()
    
    for unigram in row['unigrams']:
        # exclude n-grams with term frequency less than 3
        if unigrams_freq[unigram] >= 3:
            # Extracting “Noun"
            #'NN' is 'noun, singular or mass', 'NNS' is 'noun, plural'
            if unigram[0][1] == 'NN' or unigram[0][1] == 'NNS':
                #print(unigram, unigrams_freq[unigram])
                #print(" ")
                filtered_unigrams[unigram] = unigrams_freq[unigram]
    for bigram in row['bigrams']:
        if bigrams_freq[bigram] >= 3:
            # “Noun+Noun”
            if (bigram[0][1] == 'NN') and (bigram[1][1] == 'NN' or bigram[1][1] == 'NNS'):
                #print(bigram, bigrams_freq[bigram])
                filtered_bigrams[bigram] = bigrams_freq[bigram]
            # "Verb(past particple)+Noun"
            if (bigram[0][1] == 'VBN') and (bigram[1][1] == 'NN' or bigram[1][1] == 'NNS'):
                #print(bigram, bigrams_freq[bigram])
                filtered_bigrams[bigram] = bigrams_freq[bigram]
            # “Adjective+Noun”
            if (bigram[0][1] == 'JJ') and (bigram[1][1] == 'NN' or bigram[1][1] == 'NNS'):
                #print(bigram, bigrams_freq[bigram])
                filtered_bigrams[bigram] = bigrams_freq[bigram] 
    for trigram in row['trigrams']:
        if trigrams_freq[trigram] >= 3:
            # 'IN' is 'conjunction, subordinating or preposition', 'CC' is 'conjunction, coordinating', 'POS' is 'possessive ending'
            if (trigram[0][1] == 'NN' or trigram[0][1] == 'NNS' or trigram[0][1] == 'JJ' or trigram[0][1] == 'VB') and \
            (trigram[1][1] == 'NN' or trigram[1][1] == 'JJ' or trigram[1][1] == 'VB' or trigram[1][1] == 'IN' or trigram[1][1] == 'CC' or trigram[1][1] == 'POS') and \
            (trigram[2][1] == 'NN' or trigram[2][1] == 'NNS' or trigram[2][1] == 'JJ' or trigram[2][1] == 'VB'):
                #print(trigram, trigrams_freq[trigram])
                filtered_trigrams[trigram] = trigrams_freq[trigram]
    for quandrigram in row['quandrigrams']:
        if quandrigrams_freq[quandrigram] >= 3:
            # 'IN' is 'conjunction, subordinating or preposition', 'CC' is 'conjunction, coordinating', 'DT' is 'determiner'
            if (quandrigram[0][1] == 'NN' or quandrigram[0][1] == 'NNS' or quandrigram[0][1] == 'JJ' or quandrigram[0][1] == 'VB') and \
            (quandrigram[1][1] == 'NN' or quandrigram[1][1] == 'JJ' or quandrigram[1][1] == 'VB' or quandrigram[1][1] == 'IN' or quandrigram[1][1] == 'CC') and \
            (quandrigram[2][1] == 'NN' or quandrigram[2][1] == 'JJ' or quandrigram[2][1] == 'VB' or quandrigram[2][1] == 'IN' or quandrigram[2][1] == 'CC' or quandrigram[2][1] == 'DT') and \
            (quandrigram[3][1] == 'NN' or quandrigram[3][1] == 'NNS' or quandrigram[3][1] == 'JJ' or quandrigram[3][1] == 'VB'):
                #print(quandrigram, quandrigrams_freq[quandrigram])
                filtered_quandrigrams[quandrigram] = quandrigrams_freq[quandrigram]
    
    unigrams_grouped = group_ngrams(filtered_unigrams)
    bigrams_grouped = group_ngrams(filtered_bigrams)
    trigrams_grouped = group_ngrams(filtered_trigrams)
    quandrigrams_grouped = group_ngrams(filtered_quandrigrams)
    
    medcon_ngrams_filtered.append(OrderedDict({"filtered_unigrams": filtered_unigrams,
                                               "filtered_bigrams": filtered_bigrams,
                                               "filtered_trigrams": filtered_trigrams,
                                               "filtered_quandrigrams": filtered_quandrigrams,
                                               "unigrams_grouped": unigrams_grouped,
                                               "bigrams_grouped": bigrams_grouped,
                                               "trigrams_grouped": trigrams_grouped,
                                               "quandrigrams_grouped": quandrigrams_grouped
                                              }))

medcon_ngrams_filtered = pd.DataFrame(medcon_ngrams_filtered)
medcon_df_filtered = pd.concat([medcon_sentences, medcon_ngrams_filtered], axis=1)

      



0it [00:00, ?it/s]

3it [00:00, 29.96it/s]

72it [00:00, 40.36it/s]

639it [00:00, 57.48it/s]

1179it [00:00, 81.74it/s]

1733it [00:00, 116.04it/s]

2268it [00:00, 164.24it/s]

2814it [00:00, 231.64it/s]

3361it [00:01, 325.01it/s]

3904it [00:01, 452.68it/s]

4442it [00:01, 624.15it/s]

4987it [00:01, 849.86it/s]

5526it [00:01, 1137.16it/s]

6062it [00:01, 1488.92it/s]

6610it [00:01, 1905.16it/s]

7162it [00:01, 2370.40it/s]

7702it [00:01, 2840.56it/s]

8245it [00:01, 3313.84it/s]

8792it [00:02, 3757.69it/s]

9333it [00:02, 4112.74it/s]

9870it [00:02, 4419.16it/s]

10407it [00:02, 4635.59it/s]

10955it [00:02, 4859.50it/s]

11499it [00:02, 5017.61it/s]

12038it [00:02, 5109.79it/s]

12576it [00:02, 5187.22it/s]

13119it [00:02, 5256.33it/s]

13663it [00:02, 5308.49it/s]

14204it [00:03, 5326.16it/s]

14744it [00:03, 5327.42it/s]

15282it [00:03, 5328.52it/s]

15819it [00:03, 5321.48it/s]

16356it [00:03, 5334.12it/s]

16896it [00:03, 5353.58it/s]

17435it [00:03, 5362.70it/s]


233798it [00:52, 5173.35it/s]

234318it [00:52, 5179.28it/s]

234837it [00:52, 5134.68it/s]

235351it [00:52, 5066.61it/s]

235873it [00:52, 5108.93it/s]

236385it [00:52, 5057.60it/s]

236892it [00:52, 5056.56it/s]

237401it [00:52, 5065.26it/s]

237908it [00:53, 4971.63it/s]

238433it [00:53, 5049.77it/s]

238958it [00:53, 5106.66it/s]

239493it [00:53, 5175.52it/s]

240012it [00:53, 5151.31it/s]

240556it [00:53, 5232.06it/s]

241085it [00:53, 5247.82it/s]

241611it [00:53, 5158.30it/s]

242128it [00:53, 5158.58it/s]

242645it [00:53, 5130.30it/s]

243159it [00:54, 5118.97it/s]

243672it [00:54, 5088.03it/s]

244182it [00:54, 5078.10it/s]

244695it [00:54, 5093.06it/s]

245205it [00:54, 5037.33it/s]

245709it [00:54, 5009.19it/s]

246211it [00:54, 4992.40it/s]

246714it [00:54, 5001.52it/s]

247226it [00:54, 5034.69it/s]

247730it [00:54, 5020.51it/s]

248233it [00:55, 5007.65it/s]

248745it [00:55, 5040.70it/s]

249264it [00:55, 5082.87it/s]

249773it [00:55, 5082.34it/s]

250282it

In [19]:
medcon_df_filtered

,source_page,sent_id,sentence,filtered_unigrams,filtered_bigrams,filtered_trigrams,filtered_quandrigrams,unigrams_grouped,bigrams_grouped,trigrams_grouped,quandrigrams_grouped
0,Autism,1,"Autism is a developmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.","{(('autism', 'NN'),): 575, (('disorder', 'NN'),): 11386, (('difficulties', 'NNS'),): 883, (('interaction', 'NN'),): 301, (('communication', 'NN'),): 552, (('behavior', 'NN'),): 1179}","{(('developmental', 'JJ'), ('disorder', 'NN')): 47, (('social', 'JJ'), ('interaction', 'NN')): 57, (('repetitive', 'JJ'), ('behavior', 'NN')): 17}","{(('difficulties', 'NNS'), ('with', 'IN'), ('social', 'JJ')): 3, (('interaction', 'NN'), ('and', 'CC'), ('communication', 'NN')): 7}","{(('social', 'JJ'), ('interaction', 'NN'), ('and', 'CC'), ('communication', 'NN')): 7}","{'autism': 575, 'disorder': 11386, 'difficulties': 883, 'interaction': 301, 'communication': 552, 'behavior': 1179}","{'developmental disorder': 47, 'social interaction': 57, 'repetitive behavior': 17}","{'difficulties with social': 3, 'interaction and communication': 7}",{'social interaction and communication': 7}
1,Autism,2,Autism's symptoms result from maturation-related changes in various systems of the brain.,"{(('autism', 'NN'),): 575, (('symptoms', 'NNS'),): 19658, (('changes', 'NNS'),): 3171, (('systems', 'NNS'),): 919, (('brain', 'NN'),): 6854}","{(('various', 'JJ'), ('systems', 'NNS')): 5}",{},{},"{'autism': 575, 'symptoms': 19658, 'changes': 3171, 'systems': 919, 'brain': 6854}",{'various systems': 5},{},{}
2,Autism,3,"Some studies have reported diagnoses of autism in children due to a loss of language or social skills, as opposed to a failure to make progress, typically from 15 to 30 months of age.","{(('studies', 'NNS'),): 5061, (('diagnoses', 'NNS'),): 472, (('autism', 'NN'),): 575, (('children', 'NNS'),): 7935, (('loss', 'NN'),): 6738, (('language', 'NN'),): 1124, (('skills', 'NNS'),): 583, (('failure', 'NN'),): 3369, (('progress', 'NN'),): 346, (('months', 'NNS'),): 3177, (('age', 'NN'),): 6394}","{(('social', 'JJ'), ('skills', 'NNS')): 80}","{(('diagnoses', 'NNS'), ('of', 'IN'), ('autism', 'NN')): 3, (('loss', 'NN'), ('of', 'IN'), ('language', 'NN')): 7, (('months', 'NNS'), ('of', 'IN'), ('age', 'NN')): 154}",{},"{'studies': 5061, 'diagnoses': 472, 'autism': 575, 'children': 7935, 'loss': 6738, 'language': 1124, 'skills': 583, 'failure': 3369, 'progress': 346, 'months': 3177, 'age': 6394}",{'social skills': 80},"{'diagnoses of autism': 3, 'loss of language': 7, 'months of age': 154}",{}
3,Autism,4,No known medication relieves autism's core symptoms of social and communication impairments.,"{(('medication', 'NN'),): 1962, (('autism', 'NN'),): 575, (('core', 'NN'),): 282, (('symptoms', 'NNS'),): 19658, (('communication', 'NN'),): 552, (('impairments', 'NNS'),): 365}","{(('core', 'NN'), ('symptoms', 'NNS')): 11, (('communication', 'NN'), ('impairments', 'NNS')): 3}","{(('symptoms', 'NNS'), ('of', 'IN'), ('social', 'JJ')): 6, (('social', 'JJ'), ('and', 'CC'), ('communication', 'NN')): 5}",{},"{'medication': 1962, 'autism': 575, 'core': 282, 'symptoms': 19658, 'communication': 552, 'impairments': 365}","{'core symptoms': 11, 'communication impairments': 3}","{'symptoms of social': 6, 'social and communication': 5}",{}
4,Autism,5,"Treatment approaches have little empirical support in quality-of-life contexts, and many programs focus on success measures that lack predictive validity and real-world relevance.","{(('treatment', 'NN'),): 17805, (('approaches', 'NNS'),): 421, (('support', 'NN'),): 1085, (('contexts', 'NN'),): 55, (('programs', 'NNS'),): 453, (('success', 'NN'),): 597, (('measures', 'NNS'),): 860, (('validity', 'NN'),): 84, (('relevance', 'NN'),): 48}","{(('treatment', 'NN'), ('approaches', 'NNS')): 27, (('empirical', 'JJ'), ('support', 'NN')): 11, (('predictive', 'JJ'), ('validity', 'NN')): 3}",{},{},"{'treatment': 17805, 

In [20]:
medcon_df_filtered.to_csv('./sentences_and_ngrams/medcon_sentences_ngrams_filtered.csv', sep=';', index=False)